In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import ast
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

In [2]:
df = pd.read_csv('tra1.csv')

In [3]:
df = df.drop_duplicates()
df

,cdr3,j.segm,mhc.a,mhc.b,mhc.class,antigen.epitope,vdjdb.score,v.seg,mhc.b_label,crd3_encode,...,j.segm_3,j.segm_4,j.segm_5,mhc.a_0,mhc.a_1,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6
0,CALTPISDGQKLLF,NaN,HLA-A*01:01,B2M,0,ATDALMTGF,0,TRAV9-2*01,0,"[-0.10828808695077896, 0.42223408818244934, -0...",...,NaN,NaN,NaN,0,0,0,0,0,0,1
1,CAESTGGAGKSTF,TRAJ27*01,HLA-A*01:01,B2M,0,LTDEMIAQY,0,TRAV5*01,0,"[-0.9102591276168823, 0.584434986114502, -0.02...",...,0.0,0.0,1.0,0,0,0,0,0,0,1
2,CAGRNAGKSTF,TRAJ27*01,HLA-A*01:01,B2M,0,LTDEMIAQY,0,TRAV35*01,0,"[-0.716613233089447, -0.14133188128471375, 0.4...",...,0.0,0.0,1.0,0,0,0,0,0,0,1
3,CAVKGTNAGKSTF,TRAJ27*01,HLA-A*01:01,B2M,0,LTALRLCAY,0,TRAV12-2*01,0,"[-0.6746253371238708, 0.06916708499193192, -0....",...,0.0,0.0,1.0,0,0,0,0,0,0,1
4,CAVAFAGKLIF,TRAJ37*01,HLA-A*01:01,B2M,0,AMDEFIERY,0,TRAV21*01,0,"[-1.0499508380889893, 1.0888346433639526, 0.59...",...,0.0,1.0,1.0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35637,CLVGSSFNQGGKLIF,TRAJ23*01,HLA-E*01:03,B2M,0,RMYSPVSI,3,TRAV4*01,0,"[-2.0106520652770996, 0.8463955521583557, -0.0...",...,1.0,0.0,1.0,1,0,1,1,1,1,0
35638,CAVTNQAGTALIF,TRAJ15*01,HLA-E*01:03,B2M,0,RLPAKAPLL,3,TRAV21*01,0,"[-0.6635329127311707, 0.5018855929374695, 0.40...",...,1.0,1.0,1.0,1,0,1,1,1,1,0
35639,CAVTNQAGTALIF,TRAJ15*01,HLA-E*01:03,B2M,0,RLPAKAPLLGCG,3,TRAV21*01,0,"[-0.6635329127311707, 0.5018855929374695, 0.40...",...,1.0,1.0,1.0,1,0,1,1,1,1,0
35640,CAVTNQAGTALIF,TRAJ15*01,HLA-E*01:03,B2M,0,RLPAKAPL,3,TRAV21*01,0,"[-0.6635329127311707, 0.5018855929374695, 0.40...",...,1.0,1.0,1.0,1,0,1,1,1,1,0


In [4]:
value_counts = df['antigen.epitope'].value_counts(normalize=True)

threshold = 0.01
df = df[df['antigen.epitope'].isin(value_counts[value_counts > threshold].index)]

In [5]:
grouped_df = df.groupby(['cdr3','crd3_encode', 'j.segm', 'v.seg', 'mhc.a', 'mhc.b',
                         'v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5',
                         'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3', 'j.segm_4', 'j.segm_5',
                         'mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label', 'mhc.class', 'antigen.epitope']).max()['vdjdb.score']
df = grouped_df.reset_index()
df = df.drop_duplicates()
df.to_csv('prediction_alpha.csv', index=False)

In [5]:
cdr3_kmer = list(df['crd3_encode'].apply(lambda x: ast.literal_eval(x)))
for s in cdr3_kmer:
    if len(s) < 340:
        print(s)

In [7]:
for s in cdr3_kmer:
    if len(s) != 340:
        print(s)